In [1]:
from model import GCN,SAGE,GAT
from utils import load_data

In [2]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
from torch_sparse import SparseTensor
import random

In [3]:
nhid,dropout,lr,weight_decay=160,0.5,0.1,5e-4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs=100

In [4]:
adj, features, labels, idx_train, idx_test = load_data(device=device)

In [5]:
model = GCN(nfeat=features.shape[1],
            nhid=nhid,
            nclass=labels.max().item() + 1,
            dropout=dropout
            ).to(device)
optimizer = optim.SGD(model.parameters(),
                    lr=lr, momentum=0.5)
loss=nn.NLLLoss()

In [6]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [7]:
def train(epoch):
    model.train()
    optimizer.zero_grad()
    output = model(features, adj.t())
    #print(output)
    loss_train = loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    print('Epoch: {:04d}'.format(epoch+1),
        'loss_train: {:.4f}'.format(loss_train.item()),
        'acc_train: {:.4f}'.format(acc_train.item()))

In [8]:
for epoch in range(epochs):
    train(epoch)

tensor([[-0.7118, -0.6749],
        [-0.7047, -0.6817],
        [-0.7135, -0.6732],
        ...,
        [-0.7474, -0.6417],
        [-0.6914, -0.6949],
        [-0.7067, -0.6798]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
Epoch: 0001 loss_train: 0.6935 acc_train: 0.4972
tensor([[-0.7209, -0.6661],
        [-0.7282, -0.6593],
        [-0.7037, -0.6827],
        ...,
        [-0.7347, -0.6533],
        [-0.6943, -0.6920],
        [-0.7113, -0.6753]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
Epoch: 0002 loss_train: 0.6937 acc_train: 0.4989
tensor([[-0.6934, -0.6929],
        [-0.7014, -0.6850],
        [-0.7033, -0.6831],
        ...,
        [-0.7017, -0.6847],
        [-0.6896, -0.6967],
        [-0.7069, -0.6796]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
Epoch: 0003 loss_train: 0.6933 acc_train: 0.4962
tensor([[-0.6843, -0.7021],
        [-0.6864, -0.7000],
        [-0.6876, -0.6987],
        ...,
        [-0.6984, -0.6879],
        [-0.6958, -0.6905],
        [-

In [8]:
def test():
    model.eval()
    output = model(features, adj)
    loss_test = loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
        "loss= {:.4f}".format(loss_test.item()),
        "accuracy= {:.4f}".format(acc_test.item()))

In [19]:
test()

Test set results: loss= 0.6933 accuracy= 0.4986


In [15]:
model.state_dict()

OrderedDict([('conv1.lin_l.weight',
              tensor([[-0.0401, -0.0255, -0.0288,  ...,  0.0200,  0.0075,  0.0002],
                      [-0.0316,  0.0405, -0.0290,  ...,  0.0535,  0.0559,  0.0095],
                      [-0.0136,  0.0003,  0.0015,  ..., -0.0413,  0.0537, -0.0220],
                      ...,
                      [ 0.0381,  0.0141, -0.0533,  ..., -0.0198, -0.0158, -0.0234],
                      [ 0.0227,  0.0020,  0.0008,  ..., -0.0196, -0.0303,  0.0338],
                      [-0.0481, -0.0238, -0.0099,  ..., -0.0280,  0.0139, -0.0299]],
                     device='cuda:0')),
             ('conv1.lin_l.bias',
              tensor([-0.0256,  0.0432, -0.0201, -0.0521, -0.0492, -0.0195,  0.0121, -0.0285,
                       0.0558, -0.0407,  0.0565,  0.0174, -0.0375,  0.0025, -0.0019, -0.0375,
                      -0.0350,  0.0130,  0.0186, -0.0235, -0.0170,  0.0034, -0.0489,  0.0501,
                      -0.0191,  0.0157,  0.0374,  0.0381,  0.0294,  0.0345, 